In [1]:
# prediction based on the geolat and geolon information

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter


In [2]:
train_data = pd.read_csv('dm_all.csv')

# delete all data with empty tags from test data
train_data.dropna(subset=['photoTags'], inplace=True)
# clean all geolat so the values are in the same format
train_data['photoTags'] = train_data['photoTags'].str.replace(":", '')
train_data['photoTags'] = train_data['photoTags'].str.replace("=", '')
#take tags from the column 'photoTags'
train_geolat = train_data[train_data['photoTags'].str.contains('geolat')]
print(len(train_geolat))
k = len(train_geolat)

367567


In [5]:
# we will create excel for all the data that contains geolat information 
train_geolat.to_csv("all_geo_info.csv", encoding="utf-8",index=None)

In [8]:
train_geolat = pd.read_csv('all_geo_info.csv')
list_ln = []
list_ln_true = []
list_lat = []
list_lat_true = []

for i in range(0,len(train_geolat)): #len(train_geolat)
    phototags = train_geolat.loc[i,'photoTags']    
    try:
        start = phototags.find('geolat')
        lat = int(phototags[start+6:].split()[0])
        start = phototags.find('geolon')
        lon = int(phototags[start+6:].split()[0])
        list_lat.append(lat)
        list_ln.append(lon)
        list_ln_true.append(float(train_geolat.loc[i,'longitude'] ))
        list_lat_true.append(float(train_geolat.loc[i,'latitude']))
    except: 
        print('error')

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

In [9]:
print(len(list_lat))
len(list_ln)

343273


343273

In [10]:
geo_data = pd.DataFrame(list_ln, columns=['geolon'])
geo_data['geolat'] = list_lat
geo_data['lon'] = list_ln_true
geo_data['lat'] = list_lat_true

In [11]:
geo_data.to_csv("geo_model.csv", encoding="utf-8",index=None)

In [12]:
test_data = pd.read_csv('test2_all.csv')

# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)
# clean all geolat so the values are in the same format
test_data['photoTags'] = test_data['photoTags'].str.replace(":", '')
test_data['photoTags'] = test_data['photoTags'].str.replace("=", '')
#take tags from the column 'photoTags'
test_geolat = test_data[test_data['photoTags'].str.contains('geolat')]
print(len(test_geolat))
test_geolat.to_csv("test2_geo.csv", encoding="utf-8",index=None)

756


In [24]:
# all the indexes that have values
all_ind = test_geolat.index.values

In [18]:
test_geolat = pd.read_csv('test2_geo.csv')
result_lat = []
result_lon = []
k = len(test_geolat)
for i in range(0,k):
    phototags = test_geolat.loc[i, 'photoTags']
    #print(phototags)
    try: 
        start = phototags.find('geolat')
        lat = int(phototags[start+6:].split()[0])
        value_lat = min(list_lat, key=lambda x:abs(x-lat))

        start = phototags.find('geolon')
        lon = int(phototags[start+6:].split()[0])
        value_lon = min(list_ln, key=lambda x:abs(x-lon))
        result_lat.append(value_lat)
        result_lon.append(value_lon)
    except: 
        result_lat.append(0)
        result_lon.append(0)
        print('error')



In [19]:
test_geolat['geolat_close'] = result_lat
test_geolat['geolon_close'] = result_lon
test_geolat.to_csv("test2_geo_predict.csv", encoding="utf-8",index=None)

In [28]:
# create dict 
d_lat = {}
d_lon = {}
geo_data = pd.read_csv('geo_model.csv')
d_lon = pd.Series(geo_data.lon.values, index=geo_data.geolon).to_dict()
d_lat = pd.Series(geo_data.lat.values, index=geo_data.geolat).to_dict()

In [29]:
test_geolat = pd.read_csv('test2_geo_predict.csv')
result_lat = []
result_lon = []
k = len(test_geolat)
for i in range(0,k):
    value_lat = d_lat[test_geolat.loc[i,'geolat_close']]
    value_lon = d_lon[test_geolat.loc[i,'geolon_close']]
    result_lat.append(value_lat)
    result_lon.append(value_lon)


In [30]:
test_geolat['lat_predict'] = result_lat
test_geolat['lon_predict'] = result_lon

In [31]:
test_geolat.to_csv("test2_geo_predict.csv", encoding="utf-8",index=None)

In [34]:
test_geolat = pd.read_csv('test2_geo_predict.csv')
error = ((test_geolat['lon_predict']-test_geolat['longitude'])**2 + (test_geolat['lat_predict']-test_geolat['latitude'])**2)**(0.5)
test_geolat['error'] = error
test_geolat.to_csv("test2_geo_predict.csv", encoding="utf-8",index=None)

In [35]:
radius = [0.00001, 0.0001, 0.01, 0.1, 1, 10, 100, 400]

# 1 latitude = 111 km , so we will round it to 100 km
answers = []
for el in radius: 
    answers.append(test_geolat['error'][test_geolat['error']<el].count())
accuracy = []
amount = len(test_geolat)
print(radius)
print(answers)
for i in range(0,8):
    accuracy.append(answers[i]/amount)
print(accuracy)

[1e-05, 0.0001, 0.01, 0.1, 1, 10, 100, 400]
[14, 68, 292, 355, 401, 476, 634, 756]
[0.018518518518518517, 0.089947089947089942, 0.38624338624338622, 0.46957671957671959, 0.53042328042328046, 0.62962962962962965, 0.83862433862433861, 1.0]
